## 依赖引入

In [2]:
import warnings

import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

%matplotlib inline
warnings.filterwarnings("ignore")

## 数据读取

In [3]:
train = pd.read_csv("./data/train.csv")
train_y = train[["SALE PRICE"]]
train_X = train
del train_X["SALE PRICE"]

test_X = pd.read_csv("./data/test.csv")
test_y = pd.read_csv("./data/test_groundtruth.csv")

print("train_X:", train_X.shape)
print("train_y:", train_y.shape)
print("test_X:", test_X.shape)
print("test_y:", test_y.shape)

train_X: (43064, 19)
train_y: (43064, 1)
test_X: (10767, 19)
test_y: (10767, 1)


In [4]:
train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE DATE
0,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1569,1027,R4,402 EAST 90TH STREET,5A,10128,1,0,1,-,-,1910,2,R4,2016-09-30 00:00:00
1,4,HOLLISWOOD,10 COOPS - ELEVATOR APARTMENTS,2,10538,70,D4,"87-50 204TH STREET, B42",,11423,0,0,0,-,-,1954,2,D4,2017-02-10 00:00:00
2,3,DOWNTOWN-FULTON MALL,13 CONDOS - ELEVATOR APARTMENTS,2,170,1042,R4,265 STATE STREET,910,11201,1,0,1,0,0,2014,2,R4,2017-01-25 00:00:00
3,4,FLUSHING-NORTH,13 CONDOS - ELEVATOR APARTMENTS,2,4410,1023,R4,137-11 32 AVENUE,4W,11354,1,0,1,-,-,0,2,R4,2017-03-17 00:00:00
4,3,PARK SLOPE,09 COOPS - WALKUP APARTMENTS,2C,1067,29,C6,"862 PRESIDENT STREET, 1",,11215,0,0,0,0,0,1920,2,C6,2016-09-09 00:00:00


In [5]:
num_train_samples = len(train_X)

data_X = pd.concat([train_X, test_X])

## 处理数值数据


In [6]:
# 处理无效数据，替换无效值、0 为中位数
# 定义需要处理的列
invalid_columns = ["LAND SQUARE FEET", "GROSS SQUARE FEET"]

# 遍历每一列进行处理
for column in invalid_columns:
    # 替换无效值为 0
    train[column].replace(" -  ", "0", inplace=True)
    # 转换数据类型为 int64
    train[column] = train[column].astype("int64")

    # 筛选出非 0 的行，计算均值
    valid_index = train[column] != 0
    column_mean = int(train[valid_index][column].median())
    print("Mean: ", column_mean)
    # 替换 0 值为均值
    train.loc[train[column] == 0, column] = column_mean

    # 同样的处理应用到 data_X 上
    data_X[column].replace(" -  ", "0", inplace=True)
    data_X[column] = data_X[column].astype("int64")
    data_X.loc[data_X[column] == 0, column] = column_mean
    print(data_X[column].value_counts().sort_values(ascending=False).head(10))

Mean:  2500
LAND SQUARE FEET
2500    29104
2000     2298
4000     1836
1800      740
3000      736
5000      558
2400      326
1900      304
6000      277
1600      269
Name: count, dtype: int64
Mean:  1800
GROSS SQUARE FEET
1800    27995
1440      230
1600      230
2400      225
2000      220
1200      190
1280      184
3000      167
1152      161
1224      160
Name: count, dtype: int64


## 移除无效字段


In [7]:
del data_X["APARTMENT NUMBER"]
del data_X["BUILDING CLASS AT PRESENT"]

## 处理分类数据


In [8]:
# 处理 ADDRESS 列，移除多余信息
data_X["ADDRESS"] = data_X["ADDRESS"].apply(lambda x: x.split(",")[0])
# 转换为分类变量
# 需要转换为category类型的列名列表
category_columns = [
    "BOROUGH",
    "BUILDING CLASS CATEGORY",
    "TAX CLASS AT PRESENT",
    "TAX CLASS AT TIME OF SALE",
    "NEIGHBORHOOD",
    "ZIP CODE",
    "BLOCK",
    "BUILDING CLASS AT TIME OF SALE",
]

numerical_catagory_columns = ["ADDRESS"]

# 进行类型转换
for column in category_columns:
    data_X[column] = data_X[column].astype("category")

# 对于 numerical_catagory_columns，需要转换为 category 类型，并且转换为数值
for column in numerical_catagory_columns:
    data_X[column] = data_X[column].astype("category")
    data_X[column] = data_X[column].cat.codes

# import seaborn as sns
# sns.regplot(x="ADDRESS", y="SALE PRICE", data=train)
data_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53831 entries, 0 to 10766
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   BOROUGH                         53831 non-null  category
 1   NEIGHBORHOOD                    53831 non-null  category
 2   BUILDING CLASS CATEGORY         53831 non-null  category
 3   TAX CLASS AT PRESENT            53831 non-null  category
 4   BLOCK                           53831 non-null  category
 5   LOT                             53831 non-null  int64   
 6   ADDRESS                         53831 non-null  int32   
 7   ZIP CODE                        53831 non-null  category
 8   RESIDENTIAL UNITS               53831 non-null  int64   
 9   COMMERCIAL UNITS                53831 non-null  int64   
 10  TOTAL UNITS                     53831 non-null  int64   
 11  LAND SQUARE FEET                53831 non-null  int64   
 12  GROSS SQUARE FEET      

## 处理日期数据


In [9]:
# 从 SALE DATE 中提取出年、月、日
data_X["SALE DATE"] = pd.to_datetime(data_X["SALE DATE"])
data_X["SALE YEAR"] = data_X["SALE DATE"].dt.year
data_X["SALE MONTH"] = data_X["SALE DATE"].dt.month
data_X["SALE DAY"] = data_X["SALE DATE"].dt.day
del data_X["SALE DATE"]
data_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53831 entries, 0 to 10766
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   BOROUGH                         53831 non-null  category
 1   NEIGHBORHOOD                    53831 non-null  category
 2   BUILDING CLASS CATEGORY         53831 non-null  category
 3   TAX CLASS AT PRESENT            53831 non-null  category
 4   BLOCK                           53831 non-null  category
 5   LOT                             53831 non-null  int64   
 6   ADDRESS                         53831 non-null  int32   
 7   ZIP CODE                        53831 non-null  category
 8   RESIDENTIAL UNITS               53831 non-null  int64   
 9   COMMERCIAL UNITS                53831 non-null  int64   
 10  TOTAL UNITS                     53831 non-null  int64   
 11  LAND SQUARE FEET                53831 non-null  int64   
 12  GROSS SQUARE FEET      

## 归一化数值数据


In [10]:
all_columns = data_X.columns
# 删除类别列 category_columns
one_hot_columns = [
    "BOROUGH",
    "BUILDING CLASS CATEGORY",
    "TAX CLASS AT PRESENT",
    "TAX CLASS AT TIME OF SALE",
    "NEIGHBORHOOD",
    "ZIP CODE",
    "BLOCK",
    "BUILDING CLASS AT TIME OF SALE",
]
numerical_columns = list(set(all_columns) - set(one_hot_columns))
print("numerical_columns:", numerical_columns)
scaler = StandardScaler()
data_X[numerical_columns] = scaler.fit_transform(data_X[numerical_columns])

numerical_columns: ['TOTAL UNITS', 'GROSS SQUARE FEET', 'YEAR BUILT', 'COMMERCIAL UNITS', 'LAND SQUARE FEET', 'ADDRESS', 'SALE MONTH', 'LOT', 'SALE YEAR', 'RESIDENTIAL UNITS', 'SALE DAY']


## One-Hot 编码


In [11]:
# 对于类别数据，使用 one-hot 编码
one_hot_encoded = pd.get_dummies(data_X[one_hot_columns])
one_hot_encoded.info(verbose=True, memory_usage=True)
data_X = data_X.drop(one_hot_columns, axis=1)
data_X = pd.concat([data_X, one_hot_encoded], axis=1)

<class 'pandas.core.frame.DataFrame'>
Index: 53831 entries, 0 to 10766
Data columns (total 10873 columns):
 #      Column                                                                Dtype
---     ------                                                                -----
 0      BOROUGH_1                                                             bool 
 1      BOROUGH_2                                                             bool 
 2      BOROUGH_3                                                             bool 
 3      BOROUGH_4                                                             bool 
 4      BOROUGH_5                                                             bool 
 5      BUILDING CLASS CATEGORY_01 ONE FAMILY DWELLINGS                       bool 
 6      BUILDING CLASS CATEGORY_02 TWO FAMILY DWELLINGS                       bool 
 7      BUILDING CLASS CATEGORY_03 THREE FAMILY DWELLINGS                     bool 
 8      BUILDING CLASS CATEGORY_04 TAX CLASS 1 CONDOS

# Data Pre Processing


In [12]:
# 重新从 data_X 中分离出 train_X 和 test_X
train_X = data_X[:num_train_samples].to_numpy()
test_X = data_X[num_train_samples:].to_numpy()

# Regression


In [13]:
# 使用梯度提升决策树（GBDT）来训练模型
lgb_train = lgb.Dataset(train_X, train_y)
lgb_eval = lgb.Dataset(test_X, test_y, reference=lgb_train)

params = {
    "num_leaves": 400,
    "feature_fraction": 0.45,
    "learning_rate": 0.05,
    "objective": "mape",
}

lgb_t = lgb.train(params=params, train_set=lgb_train, num_boost_round=2000)
y_pred = lgb_t.predict(test_X)

mean_absolute_percentage_error(test_y, y_pred)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.071053 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2835
[LightGBM] [Info] Number of data points in the train set: 43064, number of used features: 802
[LightGBM] [Info] Start training from score 380000.000000


0.309445619254423

In [14]:
pd.DataFrame({"pred": y_pred}).to_csv("2110306206_Arthals.csv")